<img src='images/scipp-logo.png' width="400" height="400" style="display: block; margin-left: auto; margin-right: auto; width: 640px;">

<div style="text-align: center;">Multi-dimensional data arrays with labeled dimensions</div>
<br>
<br>

**Jan-Lukas Wynen** (<i class="fa-solid fa-envelope"></i> jan-lukas.wynen@ess.eu)
<br>

https://scipp.github.io/

# Data Reduction at ESS

<img src='images/software-stack.svg' style="display: block; margin-left: auto; margin-right: auto; width: 75%;">

### Developed at ESS w/ in-kind from ISIS

Most recent contributors:

| ESS |&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;| ISIS / Tessella |
|-----||------|
| Simon Heybrock  || Matthew Andrews |
| Neil Vaytet || Own Arnold |
| Jan-Lukas Wynen || Matthew Jones |
| || Samuel Jones |
| || Dan Nixon |
| || Tom Willemsen |

And More ...

### The problem with numpy (1)

Which dimension is which?

In [1]:
import numpy as np
a = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
a = np.r_[np.tile(a[:-1].transpose() + np.array([10, 20]), 2),
          np.array([[0, 0, 0, 1]])]
a

array([[11, 24, 11, 24],
       [12, 25, 12, 25],
       [13, 26, 13, 26],
       [ 0,  0,  0,  1]])

In [2]:
a[0]

array([11, 24, 11, 24])

Or like this?

In [3]:
a[:, 0]

array([11, 12, 13,  0])

### Scipp's solution: Labeled dimensions

In [4]:
import scipp as sc
v = sc.array(dims=['x', 'y'], values=[[1, 2], [3, 4]])
v

<scipp.Variable> (x: 2, y: 2)      int64  [dimensionless]  [1, 2, 3, 4]

In [5]:
v['x', 0]

<scipp.Variable> (y: 2)      int64  [dimensionless]  [1, 2]

In [6]:
a = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
a = np.r_[np.tile(a[:-1].transpose() + np.array([10, 20]), 2),
          np.array([[0, 0, 0, 1]])]
a

array([[11, 24, 11, 24],
       [12, 25, 12, 25],
       [13, 26, 13, 26],
       [ 0,  0,  0,  1]])

In [7]:
v = sc.array(dims=['x', 'y'], values=[[1, 2, 3], [4, 5, 6], [7, 8, 9]])
v = v['x', :-1].transpose() + sc.array(dims=['x'], values=[10, 20])
v = sc.concat([sc.concat([v, v], 'x'),
               sc.array(dims=['x'], values=[0, 0, 0, 1])],
              'y')
v.values

array([[11, 24, 11, 24],
       [12, 25, 12, 25],
       [13, 26, 13, 26],
       [ 0,  0,  0,  1]])

In [38]:
v['x', 0].values

array([11, 12, 13,  0])

### The problem with numpy (2)

How are arrays associated?

In [42]:
time = np.array([0, 1, 2, 3])
speed = np.array([0.1, 0.5, 1.3, 0.7])

Is this valid?

In [43]:
np.sum(time * speed)

5.199999999999999

### Scipp's solution: Data Arrays

In [44]:
speed_vs_time = sc.DataArray(sc.array(dims=['time'], values=speed, unit='m/s'),
                             coords={
                                 'time': sc.array(dims=['time'], values=time, unit='min')
                             })
sc.table(speed_vs_time)

In [45]:
speed_vs_time2 = sc.DataArray(sc.array(dims=['time'], values=speed, unit='m/s'),
                              coords={
                                  'time': sc.array(dims=['time'], values=time, unit='min'),
                                  'position': sc.array(dims=['time'], values=[-2, -1, 1, 1.5], unit='m')
                              })
sc.table(speed_vs_time2)

### Physcal units prevent mistakes

In [11]:
speed

<scipp.DataArray>
Dimensions: Sizes[time:4, ]
Coordinates:
  time                        int64            [min]  (time)  [0, 1, 2, 3]
Data:
                            float64            [m/s]  (time)  [0.1, 0.5, 1.3, 0.7]

Total distance:

In [12]:
sc.sum(speed.data * speed.coords['time'])

<scipp.Variable> ()    float64            [60m]  [5.2]

### Wait, isn't this just xarray?

Sort of, but scipp has

- builtin physical units <font color='#555'> &nbsp; (xarray via pint)</font>
- variances
- non-destructive masks
- bin-edge coordiantes
- binned data

### What about pandas?

`scipp.DataArray` similar to `pandas.DataFrame` but multi-dimensional

In [58]:
da = sc.DataArray(sc.array(dims=['x', 'y'], values=np.random.normal(size=[5, 10]), unit='m'),
                  coords={'x': sc.arange('x', 5),
                          'y': sc.arange('y', 4, 14, unit='s')})
da

<scipp.DataArray>
Dimensions: Sizes[x:5, y:10, ]
Coordinates:
  x                           int64  [dimensionless]  (x)  [0, 1, ..., 3, 4]
  y                           int64              [s]  (y)  [4, 5, ..., 12, 13]
Data:
                            float64              [m]  (x, y)  [-0.279754, 0.055954, ..., -0.8376, -1.06916]

In [53]:
sc.show(da)

### Coordinates prevent mistakes

In [54]:
da

<scipp.DataArray>
Dimensions: Sizes[x:5, y:10, ]
Coordinates:
  x                           int64              [m]  (x)  [0, 1, ..., 3, 4]
  y                           int64  [dimensionless]  (y)  [4, 5, ..., 12, 13]
Data:
                            float64              [m]  (x, y)  [0.746411, -0.24205, ..., -1.14692, 0.717767]

In [59]:
da2 = sc.DataArray(sc.ones(dims=['x'], shape=[5], unit='m'),
                   coords={'x': sc.arange('x', 1, 6)})
da + da2

CoordError: Mismatch in coordinate 'x' in operation 'add':
(x: 5)      int64  [dimensionless]  [0, 1, ..., 3, 4]
vs
(x: 5)      int64  [dimensionless]  [1, 2, ..., 4, 5]

### Attributes: unchecked coordinates

In [60]:
da_attr = da.copy()
da_attr.attrs['x'] = da_attr.coords.pop('x')
da_attr

<scipp.DataArray>
Dimensions: Sizes[x:5, y:10, ]
Coordinates:
  y                           int64              [s]  (y)  [4, 5, ..., 12, 13]
Data:
                            float64              [m]  (x, y)  [-0.279754, 0.055954, ..., -0.8376, -1.06916]
Attributes:
  x                           int64  [dimensionless]  (x)  [0, 1, ..., 3, 4]

In [61]:
da_attr + da2

<scipp.DataArray>
Dimensions: Sizes[x:5, y:10, ]
Coordinates:
  x                           int64  [dimensionless]  (x)  [1, 2, ..., 4, 5]
  y                           int64              [s]  (y)  [4, 5, ..., 12, 13]
Data:
                            float64              [m]  (x, y)  [0.720246, 1.05595, ..., 0.1624, -0.0691603]

### Masks: Ignore elements without removing them 

In [62]:
masked = da.copy()
masked.masks['my_mask'] = sc.array(dims=['x'], values=[True, False, False, True, False])
masked

<scipp.DataArray>
Dimensions: Sizes[x:5, y:10, ]
Coordinates:
  x                           int64  [dimensionless]  (x)  [0, 1, ..., 3, 4]
  y                           int64              [s]  (y)  [4, 5, ..., 12, 13]
Data:
                            float64              [m]  (x, y)  [-0.279754, 0.055954, ..., -0.8376, -1.06916]
Masks:
  my_mask                      bool           [None]  (x)  [True, False, ..., True, False]

In [63]:
masked.sum()

<scipp.DataArray>
Dimensions: Sizes[]
Data:
                            float64              [m]  ()  [-2.42521]

In [64]:
da.sum()

<scipp.DataArray>
Dimensions: Sizes[]
Data:
                            float64              [m]  ()  [-6.49919]

### Plotting

In [65]:
masked.plot()

# Questions?

Empty cell to show stuff

## Binned Data

In [23]:
binned = sc.data.binned_x(nevent=100, nbin=4)
binned

<scipp.DataArray>
Dimensions: Sizes[x:4, ]
Coordinates:
  x                         float64              [m]  (x [bin-edge])  [0, 0.25, ..., 0.75, 1]
Data:
                          DataArrayView           [None]  (x)  [<scipp.DataArray>
Dimensions: Sizes[row:25, ]
Coordinates:
  x                         float64              [m]  (row)  [0.118091, 0.241766, ..., 0.0338069, 0.0670407]
  y                         float64              [m]  (row)  [0.76057, 0.687262, ..., 0.00843672, 0.266437]
  z                         float64              [m]  (row)  [0.0989363, 0.553227, ..., 0.367852, 0.26365]
Data:
                            float64              [K]  (row)  [1.07883, 1.0541, ..., 1.0173, 1.05255]

, <scipp.DataArray>
Dimensions: Sizes[row:29, ]
Coordinates:
  x                         float64              [m]  (row)  [0.380196, 0.261692, ..., 0.461613, 0.350609]
  y                         float64              [m]  (row)  [0.619161, 0.174846, ..., 0.221658, 0.220269]
  z                         float64              [m]  (row)  [0.00207108, 0.806809, ..., 0.351097, 0.00326209]
Data:
                            float64              [K]  (row)  [1.0491, 1.04638, ..., 1.08232, 1.08752]

, <scipp.DataArray>
Dimensions: Sizes[row:18, ]
Coordinates:
  x                         float64              [m]  (row)  [0.609871, 0.674881, ..., 0.637122, 0.681176]
  y                         float64              [m]  (row)  [0.929865, 0.202017, ..., 0.120937, 0.568614]
  z                         float64              [m]  (row)  [0.599413, 0.911676, ..., 0.849989, 0.579709]
Data:
                            float64              [K]  (row)  [1.01625, 1.04679, ..., 1.00841, 1.0093]

, <scipp.DataArray>
Dimensions: Sizes[row:28, ]
Coordinates:
  x                         float64              [m]  (row)  [0.9767, 0.923246, ..., 0.823521, 0.947199]
  y                         float64              [m]  (row)  [0.29784, 0.301757, ..., 0.741137, 0.941554]
  z                         float64              [m]  (row)  [0.747273, 0.809356, ..., 0.454869, 0.313959]
Data:
                            float64              [K]  (row)  [1.05724, 1.07197, ..., 1.01219, 1.03879]

]

In [24]:
sc.show(binned)

### From bins to histogram

In [25]:
histogram = binned.bins.sum()
histogram

<scipp.DataArray>
Dimensions: Sizes[x:4, ]
Coordinates:
  x                         float64              [m]  (x [bin-edge])  [0, 0.25, ..., 0.75, 1]
Data:
                            float64              [K]  (x)  [26.2165, 30.7488, 18.8048, 29.326]

In [26]:
sc.show(histogram)

### Multi-dimensional bins

In [27]:
binned_2d = sc.data.binned_xy(nevent=100, nx=3, ny=4)
binned_2d

<scipp.DataArray>
Dimensions: Sizes[x:3, y:4, ]
Coordinates:
  x                         float64              [m]  (x [bin-edge])  [0, 0.333333, 0.666667, 1]
  y                         float64              [m]  (y [bin-edge])  [0, 0.25, ..., 0.75, 1]
Data:
                          DataArrayView           [None]  (x, y)  [<scipp.DataArray>
Dimensions: Sizes[row:12, ]
Coordinates:
  x                         float64              [m]  (row)  [0.261692, 0.318534, ..., 0.053002, 0.0338069]
  y                         float64              [m]  (row)  [0.174846, 0.248155, ..., 0.0148797, 0.00843672]
  z                         float64              [m]  (row)  [0.806809, 0.762911, ..., 0.960267, 0.367852]
Data:
                            float64              [K]  (row)  [1.04638, 1.09463, ..., 1.06143, 1.0173]

, <scipp.DataArray>
Dimensions: Sizes[row:3, ]
Coordinates:
  x                         float64              [m]  (row)  [0.319097, 0.26365, 0.0670407]
  y                         float64              [m]  (row)  [0.495505, 0.315656, 0.266437]
  z                         float64              [m]  (row)  [0.0545327, 0.682471, 0.26365]
Data:
                            float64              [K]  (row)  [1.08511, 1.04214, 1.05255]

, ..., <scipp.DataArray>
Dimensions: Sizes[row:13, ]
Coordinates:
  x                         float64              [m]  (row)  [0.964079, 0.671238, ..., 0.681176, 0.823521]
  y                         float64              [m]  (row)  [0.622047, 0.692759, ..., 0.568614, 0.741137]
  z                         float64              [m]  (row)  [0.730931, 0.370529, ..., 0.579709, 0.454869]
Data:
                            float64              [K]  (row)  [1.06464, 1.07175, ..., 1.0093, 1.01219]

, <scipp.DataArray>
Dimensions: Sizes[row:7, ]
Coordinates:
  x                         float64              [m]  (row)  [0.863758, 0.870415, ..., 0.808831, 0.947199]
  y                         float64              [m]  (row)  [0.948835, 0.95134, ..., 0.913138, 0.941554]
  z                         float64              [m]  (row)  [0.0280613, 0.17809, ..., 0.0039088, 0.313959]
Data:
                            float64              [K]  (row)  [1.02456, 1.05888, ..., 1.04725, 1.03879]

]

In [28]:
sc.show(binned_2d)

### Making binned data

In [29]:
events = sc.data.table_xyz(100)
events

<scipp.DataArray>
Dimensions: Sizes[row:100, ]
Coordinates:
  x                         float64              [m]  (row)  [0.9767, 0.380196, ..., 0.947199, 0.0670407]
  y                         float64              [m]  (row)  [0.29784, 0.619161, ..., 0.941554, 0.266437]
  z                         float64              [m]  (row)  [0.747273, 0.00207108, ..., 0.313959, 0.26365]
Data:
                            float64              [K]  (row)  [1.05724, 1.0491, ..., 1.03879, 1.05255]

In [30]:
x_edges = sc.linspace('x', 0.0, 1.0, 4, unit='m')
binned = sc.bin(events, edges=[x_edges])
sc.show(binned)

### Changing binning

In [31]:
binned

<scipp.DataArray>
Dimensions: Sizes[x:3, ]
Coordinates:
  x                         float64              [m]  (x [bin-edge])  [0, 0.333333, 0.666667, 1]
Data:
                          DataArrayView           [None]  (x)  [<scipp.DataArray>
Dimensions: Sizes[row:37, ]
Coordinates:
  x                         float64              [m]  (row)  [0.261692, 0.319097, ..., 0.0338069, 0.0670407]
  y                         float64              [m]  (row)  [0.174846, 0.495505, ..., 0.00843672, 0.266437]
  z                         float64              [m]  (row)  [0.806809, 0.0545327, ..., 0.367852, 0.26365]
Data:
                            float64              [K]  (row)  [1.04638, 1.08511, ..., 1.0173, 1.05255]

, <scipp.DataArray>
Dimensions: Sizes[row:26, ]
Coordinates:
  x                         float64              [m]  (row)  [0.380196, 0.441006, ..., 0.461613, 0.350609]
  y                         float64              [m]  (row)  [0.619161, 0.277216, ..., 0.221658, 0.220269]
  z                         float64              [m]  (row)  [0.00207108, 0.762591, ..., 0.351097, 0.00326209]
Data:
                            float64              [K]  (row)  [1.0491, 1.02263, ..., 1.08232, 1.08752]

, <scipp.DataArray>
Dimensions: Sizes[row:37, ]
Coordinates:
  x                         float64              [m]  (row)  [0.9767, 0.923246, ..., 0.823521, 0.947199]
  y                         float64              [m]  (row)  [0.29784, 0.301757, ..., 0.741137, 0.941554]
  z                         float64              [m]  (row)  [0.747273, 0.809356, ..., 0.454869, 0.313959]
Data:
                            float64              [K]  (row)  [1.05724, 1.07197, ..., 1.01219, 1.03879]

]

In [32]:
fine_x_edges = sc.linspace('x', 0.0, 1.0, 12, unit='m')
sc.bin(binned, edges=[fine_x_edges])

<scipp.DataArray>
Dimensions: Sizes[x:11, ]
Coordinates:
  x                         float64              [m]  (x [bin-edge])  [0, 0.0909091, ..., 0.909091, 1]
Data:
                          DataArrayView           [None]  (x)  [<scipp.DataArray>
Dimensions: Sizes[row:11, ]
Coordinates:
  x                         float64              [m]  (row)  [0.0601387, 0.0601373, ..., 0.0338069, 0.0670407]
  y                         float64              [m]  (row)  [0.728827, 0.556402, ..., 0.00843672, 0.266437]
  z                         float64              [m]  (row)  [0.0469184, 4.61286e-05, ..., 0.367852, 0.26365]
Data:
                            float64              [K]  (row)  [1.00044, 1.09189, ..., 1.0173, 1.05255]

, <scipp.DataArray>
Dimensions: Sizes[row:6, ]
Coordinates:
  x                         float64              [m]  (row)  [0.118091, 0.172066, ..., 0.13441, 0.136524]
  y                         float64              [m]  (row)  [0.76057, 0.127554, ..., 0.981099, 0.72708]
  z                         float64              [m]  (row)  [0.0989363, 0.98037, ..., 0.462533, 0.049891]
Data:
                            float64              [K]  (row)  [1.07883, 1.01681, ..., 1.06678, 1.01892]

, ..., <scipp.DataArray>
Dimensions: Sizes[row:13, ]
Coordinates:
  x                         float64              [m]  (row)  [0.863621, 0.863758, ..., 0.873824, 0.823521]
  y                         float64              [m]  (row)  [0.279595, 0.948835, ..., 0.216893, 0.741137]
  z                         float64              [m]  (row)  [0.235477, 0.0280613, ..., 0.432808, 0.454869]
Data:
                            float64              [K]  (row)  [1.03548, 1.02456, ..., 1.08424, 1.01219]

, <scipp.DataArray>
Dimensions: Sizes[row:12, ]
Coordinates:
  x                         float64              [m]  (row)  [0.9767, 0.923246, ..., 0.956514, 0.947199]
  y                         float64              [m]  (row)  [0.29784, 0.301757, ..., 0.0903527, 0.941554]
  z                         float64              [m]  (row)  [0.747273, 0.809356, ..., 0.443394, 0.313959]
Data:
                            float64              [K]  (row)  [1.05724, 1.07197, ..., 1.09963, 1.03879]

]

### Binning in additional dimension

In [33]:
y_edges = sc.geomspace('y', 0.1, 0.8, 5, unit='m')
binned_2d = sc.bin(binned, edges=[y_edges])
sc.show(binned_2d)

Equivalently:

In [34]:
sc.bin(binned, edges=[x_edges, y_edges])

<scipp.DataArray>
Dimensions: Sizes[x:3, y:4, ]
Coordinates:
  x                         float64              [m]  (x [bin-edge])  [0, 0.333333, 0.666667, 1]
  y                         float64              [m]  (y [bin-edge])  [0.1, 0.168179, ..., 0.475683, 0.8]
Data:
                          DataArrayView           [None]  (x, y)  [<scipp.DataArray>
Dimensions: Sizes[row:2, ]
Coordinates:
  x                         float64              [m]  (row)  [0.172066, 0.293502]
  y                         float64              [m]  (row)  [0.127554, 0.12208]
  z                         float64              [m]  (row)  [0.98037, 0.0448469]
Data:
                            float64              [K]  (row)  [1.01681, 1.09533]

, <scipp.DataArray>
Dimensions: Sizes[row:5, ]
Coordinates:
  x                         float64              [m]  (row)  [0.261692, 0.318534, ..., 0.280349, 0.0670407]
  y                         float64              [m]  (row)  [0.174846, 0.248155, ..., 0.191316, 0.266437]
  z                         float64              [m]  (row)  [0.806809, 0.762911, ..., 0.8454, 0.26365]
Data:
                            float64              [K]  (row)  [1.04638, 1.09463, ..., 1.08841, 1.05255]

, ..., <scipp.DataArray>
Dimensions: Sizes[row:4, ]
Coordinates:
  x                         float64              [m]  (row)  [0.9767, 0.923246, 0.735758, 0.893008]
  y                         float64              [m]  (row)  [0.29784, 0.301757, 0.380303, 0.439449]
  z                         float64              [m]  (row)  [0.747273, 0.809356, 0.723464, 0.256064]
Data:
                            float64              [K]  (row)  [1.05724, 1.07197, 1.00387, 1.06599]

, <scipp.DataArray>
Dimensions: Sizes[row:15, ]
Coordinates:
  x                         float64              [m]  (row)  [0.964079, 0.671238, ..., 0.797702, 0.823521]
  y                         float64              [m]  (row)  [0.622047, 0.692759, ..., 0.488634, 0.741137]
  z                         float64              [m]  (row)  [0.730931, 0.370529, ..., 0.275026, 0.454869]
Data:
                            float64              [K]  (row)  [1.06464, 1.07175, ..., 1.06376, 1.01219]

]

### Computation with binned data

In [35]:
dense_x = sc.DataArray(sc.array(dims=['x'], values=[1, 2, 3], unit='1/kg'),
                       coords={'x': x_edges})
dense_x

<scipp.DataArray>
Dimensions: Sizes[x:3, ]
Coordinates:
  x                         float64              [m]  (x [bin-edge])  [0, 0.333333, 0.666667, 1]
Data:
                              int64           [1/kg]  (x)  [1, 2, 3]

In [36]:
dense_x * binned_2d

<scipp.DataArray>
Dimensions: Sizes[x:3, y:4, ]
Coordinates:
  x                         float64              [m]  (x [bin-edge])  [0, 0.333333, 0.666667, 1]
  y                         float64              [m]  (y [bin-edge])  [0.1, 0.168179, ..., 0.475683, 0.8]
Data:
                          DataArrayView           [None]  (x, y)  [<scipp.DataArray>
Dimensions: Sizes[row:2, ]
Coordinates:
  x                         float64              [m]  (row)  [0.172066, 0.293502]
  y                         float64              [m]  (row)  [0.127554, 0.12208]
  z                         float64              [m]  (row)  [0.98037, 0.0448469]
Data:
                            float64           [K/kg]  (row)  [1.01681, 1.09533]

, <scipp.DataArray>
Dimensions: Sizes[row:5, ]
Coordinates:
  x                         float64              [m]  (row)  [0.261692, 0.318534, ..., 0.280349, 0.0670407]
  y                         float64              [m]  (row)  [0.174846, 0.248155, ..., 0.191316, 0.266437]
  z                         float64              [m]  (row)  [0.806809, 0.762911, ..., 0.8454, 0.26365]
Data:
                            float64           [K/kg]  (row)  [1.04638, 1.09463, ..., 1.08841, 1.05255]

, ..., <scipp.DataArray>
Dimensions: Sizes[row:4, ]
Coordinates:
  x                         float64              [m]  (row)  [0.9767, 0.923246, 0.735758, 0.893008]
  y                         float64              [m]  (row)  [0.29784, 0.301757, 0.380303, 0.439449]
  z                         float64              [m]  (row)  [0.747273, 0.809356, 0.723464, 0.256064]
Data:
                            float64           [K/kg]  (row)  [3.17172, 3.21591, 3.0116, 3.19797]

, <scipp.DataArray>
Dimensions: Sizes[row:15, ]
Coordinates:
  x                         float64              [m]  (row)  [0.964079, 0.671238, ..., 0.797702, 0.823521]
  y                         float64              [m]  (row)  [0.622047, 0.692759, ..., 0.488634, 0.741137]
  z                         float64              [m]  (row)  [0.730931, 0.370529, ..., 0.275026, 0.454869]
Data:
                            float64           [K/kg]  (row)  [3.19392, 3.21526, ..., 3.19129, 3.03658]

]

# Questions?

Empty cell to show stuff

# Tutorial

<img src='images/flares.svg' style="display: block; margin-left: auto; margin-right: auto; width: 65%;">